In [ ]:
import pandas as pd

df_original = pd.read_csv('/content/European_bank_marketing.csv')
df_original

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,term_deposit,Ethnicity_African
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0,0
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0,0
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0,0
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0,0
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,...,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,1,0
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,...,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,0,0
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,...,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,0,0
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,...,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,1,0


In [ ]:
# find categorical columns in the dataframe
cat_columns = df_original.select_dtypes(include=['object']).columns.tolist()
# find numerical columns in the dataframe
num_columns = df_original.select_dtypes(include=['number']).columns.tolist()
exclude_columns = ['Ethnicity_African','term_deposit']
num_columns = [col for col in num_columns if col not in exclude_columns]


# print the numerical columns
print(num_columns)

# print the categorical columns
print(cat_columns)

['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']
['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder
from sklearn.compose import ColumnTransformer

def poly_degree_analysis(df, target_col, numerical_cols, categorical_cols=None, degrees=[2, 3, 4, 5]):
    """
    Perform polynomial degree analysis for logistic regression model using cross-validation.

    Parameters:
    df (pandas.DataFrame): The input DataFrame.
    target_col (str): The name of the target column.
    numerical_cols (list of str): The names of the numerical columns.
    categorical_cols (list of str): The names of the categorical columns (default=None).
    degrees (list of int): The range of polynomial degrees to test (default=[2, 3, 4, 5]).

    Returns:
    results (pandas.DataFrame): A DataFrame containing the degree, mean score, and std score for each polynomial degree.
    """
    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(df.drop(target_col, axis=1), df[target_col], test_size=0.3, random_state=42)

    # Define the logistic regression model
    logreg = LogisticRegression(max_iter=10000)

    # # Define the column transformer to apply the polynomial features only to numerical columns
    # preprocessor = ColumnTransformer(
    #     transformers=[
    #         ('poly', PolynomialFeatures(include_bias=False), numerical_cols),
    #         ('cat', OneHotEncoder(drop='if_binary'), categorical_cols)
    #     ])

    # Create an empty list to store the results
    results = []

    # Loop over the range of polynomial degrees
    for degree in degrees:
        # Define the polynomial features transformer
        poly_transformer = PolynomialFeatures(degree=degree, include_bias=False)

        # Define the full preprocessor
        full_preprocessor = ColumnTransformer(
            transformers=[
                ('poly', poly_transformer, numerical_cols),
                ('cat', OneHotEncoder(drop='if_binary'), categorical_cols)
            ])

        # Apply the preprocessor to the training data
        X_train_processed = full_preprocessor.fit_transform(X_train)

        # Evaluate the performance of the model using cross-validation
        scores = cross_val_score(logreg, X_train_processed, y_train, cv=5)

        # Calculate the mean and standard deviation of the cross-validation scores
        mean_score = np.mean(scores)
        std_score = np.std(scores)

        # Add the results to the list
        results.append((degree, mean_score, std_score))

    # Create a DataFrame to store the results
    results_df = pd.DataFrame(results, columns=['degree', 'mean_score', 'std_score'])

    return results_df


In [ ]:
poly_degree_analysis(df_original,'term_deposit',num_columns,cat_columns)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/l

,degree,mean_score,std_score
0,2,0.909230,0.001486
1,3,0.909958,0.001807
2,4,0.887239,0.000062
3,5,0.887239,0.000062


It is evident from above dataframe that degree = 2 polynomial should suffice, it has high mean score for a lower standard deviation for degree=3

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

def create_poly_features(df):
    # Create an instance of the PolynomialFeatures transformer with degree 2
    poly = PolynomialFeatures(degree=2, include_bias=False)
    
    # Use the transformer to create polynomial features for all columns
    poly_features = poly.fit_transform(df)
    
    # Create a list of column names for the new polynomial features
    col_names = poly.get_feature_names_out(df.columns)
    
    # Create a new DataFrame with the polynomial features and the column names
    poly_df = pd.DataFrame(poly_features, columns=col_names)
    
    return poly_df


In [ ]:
poly_df = create_poly_features(df_original[num_columns])
poly_df

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,cons.price.idx^2,cons.price.idx cons.conf.idx,cons.price.idx euribor3m,cons.price.idx nr.employed,cons.conf.idx^2,cons.conf.idx euribor3m,cons.conf.idx nr.employed,euribor3m^2,euribor3m nr.employed,nr.employed^2
0,56.0,261.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857,5191.0,...,8834.872036,-3421.3816,456.528858,487922.8540,1324.96,-176.7948,-188952.40,23.590449,25212.6870,26946481.00
1,57.0,149.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857,5191.0,...,8834.872036,-3421.3816,456.528858,487922.8540,1324.96,-176.7948,-188952.40,23.590449,25212.6870,26946481.00
2,37.0,226.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857,5191.0,...,8834.872036,-3421.3816,456.528858,487922.8540,1324.96,-176.7948,-188952.40,23.590449,25212.6870,26946481.00
3,40.0,151.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857,5191.0,...,8834.872036,-3421.3816,456.528858,487922.8540,1324.96,-176.7948,-188952.40,23.590449,25212.6870,26946481.00
4,56.0,307.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857,5191.0,...,8834.872036,-3421.3816,456.528858,487922.8540,1324.96,-176.7948,-188952.40,23.590449,25212.6870,26946481.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73.0,334.0,1.0,999.0,0.0,-1.1,94.767,-50.8,1.028,4963.6,...,8980.784289,-4814.1636,97.420476,470385.4812,2580.64,-52.2224,-252150.88,1.056784,5102.5808,24637324.96
41184,46.0,383.0,1.0,999.0,0.0,-1.1,94.767,-50.8,1.028,4963.6,...,8980.784289,-4814.1636,97.420476,470385.4812,2580.64,-52.2224,-252150.88,1.056784,5102.5808,24637324.96
41185,56.0,189.0,2.0,999.0,0.0,-1.1,94.767,-50.8,1.028,4963.6,...,8980.784289,-4814.1636,97.420476,470385.4812,2580.64,-52.2224,-252150.88,1.056784,5102.5808,24637324.96
41186,44.0,442.0,1.0,999.0,0.0,-1.1,94.767,-50.8,1.028,4963.6,...,8980.784289,-4814.1636,97.420476,470385.4812,2580.64,-52.2224,-252150.88,1.056784,5102.5808,24637324.96


In [ ]:
def concat_df(df_list):
  # concatenate the dataframes horizontally
  df_concat = pd.concat(df_list, axis=1)

  # drop any duplicating columns
  df_concat = df_concat.loc[:, ~df_concat.columns.duplicated()]

  return df_concat


In [ ]:
df_ft_int = pd.read_csv('/content/ft_int.csv')

df_list = [df_ft_int, poly_df,df_original]
df_final = concat_df(df_list)
df_final

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,cons.price.idx^2,cons.price.idx cons.conf.idx,cons.price.idx euribor3m,cons.price.idx nr.employed,cons.conf.idx^2,cons.conf.idx euribor3m,cons.conf.idx nr.employed,euribor3m^2,euribor3m nr.employed,nr.employed^2
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,8834.872036,-3421.3816,456.528858,487922.8540,1324.96,-176.7948,-188952.40,23.590449,25212.6870,26946481.00
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,8834.872036,-3421.3816,456.528858,487922.8540,1324.96,-176.7948,-188952.40,23.590449,25212.6870,26946481.00
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,8834.872036,-3421.3816,456.528858,487922.8540,1324.96,-176.7948,-188952.40,23.590449,25212.6870,26946481.00
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,8834.872036,-3421.3816,456.528858,487922.8540,1324.96,-176.7948,-188952.40,23.590449,25212.6870,26946481.00
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,8834.872036,-3421.3816,456.528858,487922.8540,1324.96,-176.7948,-188952.40,23.590449,25212.6870,26946481.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,...,8980.784289,-4814.1636,97.420476,470385.4812,2580.64,-52.2224,-252150.88,1.056784,5102.5808,24637324.96
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,...,8980.784289,-4814.1636,97.420476,470385.4812,2580.64,-52.2224,-252150.88,1.056784,5102.5808,24637324.96
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,...,8980.784289,-4814.1636,97.420476,470385.4812,2580.64,-52.2224,-252150.88,1.056784,5102.5808,24637324.96
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,...,8980.784289,-4814.1636,97.420476,470385.4812,2580.64,-52.2224,-252150.88,1.056784,5102.5808,24637324.96


In [ ]:
# export the DataFrame as a CSV file
df_final.to_csv('mlfin.csv', index=False)